 This project contains voting data provided by [YouGov](https://www.voterstudygroup.org/publication/2019-voter-survey-full-data-set) which contains information on peronal characteristics and answers to survey questions. Most of the surveyed people participated in ealrier surveys available on the companies hompage.<br>
This project tough only focuses on the cross-section. Extensions are possible. <br>
Since the survey contains questions and personal characteristics in dimensions greater than 1000 the aim of this project is to twofold: <br>
First to reduce the dimension in order to obtain voter categories determined by a small number of parameters. <br>
Second to check wheter membership to a cretain group can reliable predict voting behavior.

In [50]:
options(messages = -1) # does not include errors and warnings
library(tidyverse)
library(data.table)
# library(glmnet) # LASSO
library(factoextra) # PCA
library(nnet)
library(ade4)

In [51]:
path_sven <- "~/06_data/VOTER_Survey_Jan217_Release1-csv.csv"
data <- read.csv(file = path_sven, header = TRUE, sep = ",")
#head(data)
#str(data)

the data set contains 1282 variables where the first 11 columns are population weights and not of interest in further analysis.

# 1. Data Cleaning

Since the dataset contains quite a lot of variables in further analysis we will focus on only a subset for reasons of tractability

In [68]:
# weights not of interest
weight_idx <- grepl(".*weight.*", colnames(data)) 
data <- data[,!weight_idx]
# name of candidate not of interest
name_idx <- grepl(".*Cand.Name.*", colnames(data)) 
data <- data[,!name_idx]
# response to july 2017 survey
survey2017_idx <- grepl(".*_2017", colnames(data)) 
data <- data[,!survey2017_idx]
#response to may 2018 survey
survey2018_idx <- grepl(".*_2018", colnames(data)) 
data <- data[,!survey2018_idx]
# cassfullcd
cassfullcd_idx <- grepl("cassfullcd", colnames(data))
data <- data[,!cassfullcd_idx]
# to be continued

# only democrat and republicans
data <- subset(data, data$vote2020_2019 == 1 |data$vote2020_2019 == 2 )

,vote2020_2019,trumpapp_2019,fav_trump_2019,fav_obama_2019,fav_hrc_2019,fav_sanders_2019,fav_putin_2019,fav_schumer_2019,fav_pelosi_2019,fav_comey_2019,⋯,phone_baseline,faminc_baseline,cand_baseline,cdid_baseline,demnom2_baseline,demnom_t_baseline,izip_baseline,lookupzip_baseline,prim12_baseline,statefips_baseline
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<fct>
1,2,4,4,1,1,1,4,1,1,1,⋯,1,6,258342,03,1,,89052,89052,1,32
2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,1,8,317635,6,3,,85298,85298,2,04
3,2,4,4,1,1,2,4,1,1,1,⋯,1,5,102825,6,2,Hillery Clinton,54904,54904,1,55
4,2,4,4,1,4,4,4,3,3,2,⋯,1,9,45897,1,1,,74104,74104,3,40
5,1,2,2,2,4,3,3,4,4,4,⋯,1,10,330742,21,3,,78148,78148,2,48
6,2,4,4,1,1,3,4,2,1,2,⋯,3,4,101855,12,1,,94061,94061,1,06


In [69]:
# response variabe of interest
# 1 -> Donald Trump
# 2 -> Dem
# 3 -> no vote
# 4 -> not sure
# 8 -> skipped

# ToDo: filter only rows where either Donald T or Dem voted for

str(data$vote2020_2019)
table(data$vote2020_2019)
y_col <- grep("vote2020_2019", colnames(data)) 
y <- data[,y_col]
y <- factor(y)
levels(y) <-  c("Donald_Trump", "Democrat", "No_Vote", "Not_Sure", "Skipped")    
data <- data[,-y_col]
col <- colnames(data)
data[col] <- lapply(data[col], as.factor) # due to survey design all variables are factor
data <- data.frame(data) # factor -> dummy 

 int [1:5896] 2 2 2 1 2 2 2 2 2 2 ...



   1    2 
2630 3266 

In [48]:
data_dummy <- acm.disjonctif(data)

# 2. 

In [41]:
set.seed(123)
n <- nrow(data)
p = ncol(data) - 1
ind_train <- sample(x = 1:n, size = ceiling(0.8 * n))
set_train <- data[ind_train,]
ind_test <- setdiff(x = 1:n, ind_train)
set_test <- data[ind_test,]

In [ ]:
# factor => dummy


# 3. Neural Net

In [42]:
nnet::nnet(x = data, y = y)

ERROR: Error in nnet.default(x = data, y = y): missing values in 'x'


# 4. LASSO

In [43]:
glmnet::glmnet(x = as.matrix(data), y = y, alpha = 1) # Elastic Net with alpha = 1

Warning message in storage.mode(xd) <- "double":
“NAs introduced by coercion”


ERROR: Error in storage.mode(y) <- "double": invalid to change the storage mode of a factor


# 5. Latent Drichilet Allocation

In [5]:
options(messages = 1)